In [1]:
from vehicle import Vehicle
import cv2
from ultralytics import YOLO
import time

model = YOLO('yolov8n.pt')

In [2]:
def Is_ID_in_vehicle_list(vehicle_list, target_id,classname,centre):
     
     for vehicle in vehicle_list:
        if vehicle.vehicle_id == target_id:
            return True,vehicle
        
     v=Vehicle(classname,target_id,centre)
     return False,v

def centre(x1, y1, x2, y2):
    center_x = int((x1 + x2) / 2)
    center_y = int((y1 + y2) / 2)
    return center_x,center_y


In [3]:
video_path = "C:/Users/Lina/Downloads/TF.mp4"

In [4]:

Allowed=['car','motorcycle''bus',  'truck']

In [5]:
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_thickness = 2
font_color = (255, 255, 255)
Rectangle_font_color=(0, 0, 255)

In [6]:
# Open the video file
cap = cv2.VideoCapture(video_path)
#Getting orginal video fps needed for speed calculation
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000 / fps)
print("delay:",delay,fps)

# Loop through the video frames
count=0
ModelNames = model.names
trues=0
crossed=[]
Vehicles=[]

delay: 33 30.0


In [18]:
count=0
cap = cv2.VideoCapture(video_path)
while cap.isOpened():
 success, frame = cap.read()
 height, width, channels = frame.shape
 bottom_height=int(height/2)
 top_height=bottom_height-100

 if not success: break
 count+=1 #counting frames
 
 if success:
   
   results= model.track(frame,persist=True)
   boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
   if results[0].boxes.id is None:continue
   ids = results[0].boxes.id.cpu().numpy().astype(int)
   names = results[0].boxes.cls.cpu().numpy().astype(int)
   j=0
   for box in boxes:
      if ModelNames[names[j]] in Allowed:

        x1, y1, x2, y2 = box
        center_x ,center_y = centre(x1, y1, x2, y2)

        Is_In,v=Is_ID_in_vehicle_list(Vehicles,ids[j],ModelNames[names[j]],center_y)
        
        if not Is_In:
            Vehicles.append(v)                
        Rectangle_font_color=(0, 0, 255)                      
        if v.top_box is None:  
           if v.FirstCentre_y>top_height and top_height >= center_y:
               v.Top_Crossed(box,count)
           elif v.FirstCentre_y<top_height and top_height <= center_y:
               v.Top_Crossed(box,count)
           
        if v.bottom_box is None:  
           if v.FirstCentre_y>bottom_height and bottom_height >= center_y:
               v.Bottom_Crossed(box,count)
               v.crop_and_save(frame)
           elif v.FirstCentre_y<bottom_height and bottom_height <= center_y:
               v.Bottom_Crossed(box,count)
               

          
        #Color black when between lines
        if top_height < center_y < bottom_height:
          Rectangle_font_color=(0, 0, 0)
        
        annotation=""
        if v.vitesse is None:
         annotation=v.calcul_vitesse(fps,12)
        else: 
           annotation=v.vitesse
           Rectangle_font_color=(255, 255, 0)
         
        j+=1
        cv2.putText(frame, annotation,(x1, y1),font, font_scale, font_color, font_thickness)
        cv2.circle(frame, (center_x, center_y), 1, (0, 0, 255), 1)
        cv2.rectangle(frame, (x1, y1), (x2, y2), Rectangle_font_color, 2) 
                 
            # Display the image with bounding boxes
   
   cv2.line(frame, (0,top_height), (width,top_height), (255,0, 0) , 1)#blue top line
   cv2.line(frame, (0,bottom_height), (width,bottom_height), (0, 255, 0) , 1)#green bottom line
   cv2.imshow("First Frame with Line", frame)
   if cv2.waitKey(delay) & 0xFF == ord("q"):
            break


print(Vehicles)
cap.release()
cv2.destroyAllWindows()




0: 384x640 11 cars, 1 truck, 343.4ms
Speed: 48.9ms preprocess, 343.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 272.3ms
Speed: 3.5ms preprocess, 272.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 265.2ms
Speed: 8.0ms preprocess, 265.2ms inference, 7.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 245.4ms
Speed: 0.0ms preprocess, 245.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 281.4ms
Speed: 2.0ms preprocess, 281.4ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 266.1ms
Speed: 3.1ms preprocess, 266.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 255.3ms
Speed: 0.0ms preprocess, 255.3ms inference, 4.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 207.2ms
Speed: 7.6ms preprocess, 207.2ms inference, 0.0ms postprocess per image at shap

In [10]:
import cv2
from ultralytics import YOLO
license_plate_detector = YOLO('license_plate_detector.pt')

In [13]:
import cv2  
frame = cv2.imread("3.png")
license_plates = license_plate_detector(frame)[0]

for license_plate in license_plates.boxes.data.tolist():
    x1, y1, x2, y2, score, class_id = license_plate
   
    license_plate_crop = frame[int(y1):int(y2), int(x1):int(x2), :]
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
 
    cv2.imwrite('c3.jpg', license_plate_crop)
cv2.imshow('Original Frame', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 448x640 1 license_plate, 463.5ms
Speed: 4.1ms preprocess, 463.5ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)
